# $orvara$ tutorial

## Please make sure you are running this notebook from within the root directory!

In [2]:
ls

build/                    HGCA_vEDR3.fits  orvara/           README.rst
CHANGES.md                LICENCE          orvara.egg-info/  setup.py
HGCA_vDR2_corrected.fits  misc/            pytest.ini        Tutorial.ipynb


### The above verifies that you are in the root directory.
You should see something like:


    build/                    HGCA_vEDR3.fits  orvara/           README.rst
    CHANGES.md                LICENCE          orvara.egg-info/  setup.py
    HGCA_vDR2_corrected.fits  misc/            pytest.ini        Tutorial.ipynb

## Now that you are in the root directory, lets install orvara.

### Warning about path names:
DO NOT run orbit fits from paths that contain files with spaces. This can cause python to fail at finding various input files. So if you install orvara and run from:

    /user/your name/Research Projects/orvara/
That may cause failures. But:

    /user/yourname/ResearchProjects/orvara/

is OK. If you are stuck with path names with spaces: You can always try first to see if it works with your directory paths, if you have spaces involved. But if you get a wierd error (especially when you the orbit fit has yet to actually start and is only trying to load in data), your path names containing spaces are probably the issue.

# Note: there is no need to run this following cell if you already have orvara installed.

In [3]:
!pip install -e .

Obtaining file:///home/gmbrandt/Documents/Repositories/orvara
  Preparing metadata (setup.py) ... done
  Attempting uninstall: orvara
    Found existing installation: orvara 1.1.2
    Uninstalling orvara-1.1.2:
      Successfully uninstalled orvara-1.1.2
  Running setup.py develop for orvara


If you see: "Successfully installed orvara-X.X.X"

With X.X.X being the version number,
Great!

## if you installed orvara with the above cell, then restart the Kernal in this jupyter notebook, then continue with the next cell.

### Run pytest to make sure that the tests pass

This will run a small suite of tests. This should take < 30 seconds. It will complete with a few warnings -- that is OK.

**If this cell hangs for a long time (more than 3 minutes)-- please skip this cell and continue.**

If any of the tests fail, something is wrong with the install. Try running "pip install -e ." again If the issue persists, please submit an issue ticket!

In [1]:
!pytest -sv -m "not e2e"

============================= test session starts ==============================
platform linux -- Python 3.8.10, pytest-6.2.3, py-1.10.0, pluggy-0.13.1 -- /home/gmbrandt/envs/orvara_new/bin/python
cachedir: .pytest_cache
rootdir: /home/gmbrandt/Documents/Repositories/orvara, configfile: pytest.ini
collected 9 items / 3 deselected / 6 selected                                  

orvara/tests/test_config.py::test_example_config_file PASSED
orvara/tests/test_main.py::test_set_initial_parameters PASSED
orvara/tests/test_main.py::test_get_priors PASSED
orvara/tests/test_main.py::test_run Loading RV data from file orvara/tests/data/Gl758_RV.dat
Loaded data from 3 RV instruments.
Loading astrometric data for 1 planets
Loaded 4 relative astrometric data points from file orvara/tests/data/Gl758_relAST.txt
Loading absolute astrometry data for Hip 95319
Recognized a 5-parameter fit in Gaia for Hip 95319
Not using companion proper motion from Gaia.
Using ptemcee.
Running MCMC ... 
[###############

### Check the usage of fit_orbit and plot_orbit

fit_orbit and plot_orbit are the key commands to operate this package. Run --h to see their functionality. 

both commands require one optional argument --output-dir immediately followed by the desired directory to write output to. fit_orbit outputs the MCMC chain from the fit and plot_orbit outputs a set of publication-quality plots. If --output-dir flag is omitted, the output directory will be automatically set to the current working directory by default. 

Following the output directory is the configuration file tailored to the system you are fitting. Example configuration files can be found in /orvara/tests/.

In [2]:
!fit_orbit --h

usage: fit_orbit [-h] [--output-dir OUTPUT_DIR] configfile

Fit a Keplerian orbit or orbits using orvara.

positional arguments:
  configfile            Path to the planet-specific configuration file.

optional arguments:
  -h, --help            show this help message and exit
  --output-dir OUTPUT_DIR
                        Directory within which to save the MCMC results,
                        default current directory.


In [3]:
!plot_orbit --h

usage: plot_orbit [-h] [--output-dir OUTPUT_DIR] configfile

Plot astrometry or RV orbits and other relavant plots.

positional arguments:
  configfile            Path to the planet-specific configuration file.

optional arguments:
  -h, --help            show this help message and exit
  --output-dir OUTPUT_DIR
                        Directory within which to save the plots, default
                        current directory.


### Running an example of fit_orbit, we will fit orbit to the White Dwarf - Main Sequence (WD-MS) binary system HD159062. 

The customizable configuration file for HD 159062 is located in orvara/tests/

So let's load the config.ini file for HD 159062 here:

In [4]:
ls orvara/tests/

chains/                                   diagnostic_config.ini
config_Gl758.ini                          plots/
config_HD159062.ini                       __pycache__/
config_HD4747.ini                         start_Gl758.dat
config_hip21.ini                          start_hip3850_after7pfit.dat
config.ini                                start_hip3850.dat
config_with_secondary_priors.ini          test_config.py
data/                                     test_main.py
diagnostic_config_fake_fulldr3_7pfit.ini  utils.py
diagnostic_config_fake_fulldr3_9pfit.ini


In [11]:
def display_file(fpath):
    with open(fpath, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.rstrip('\n')
            print(line)

In [13]:
print('This is what an orvara configuration file looks like:')
display_file('orvara/tests/config_HD159062.ini')

This is what an orvara configuration file looks like:
[data_paths]
# Hipparcos ID of the star, used for fetching absolute astrometry.
# If the Hipparcos ID is invalid the fit will not use HGCA astrometry. 
# In that case you must supply a parallax and parallax_error in [priors].
HipID = 85653
# The file containing the Hipparcos Gaia Catalog of Accelerations.
HGCAFile = HGCA_vEDR3_withchi2.fits
# The file containing the radial velocity time series for the star.
RVFile = orvara/tests/data/HD159062_RV.dat
# The file containing the relative astrometry for the star.
AstrometryFile = orvara/tests/data/HD159062_relAST.txt
# The path to the Gaia/GOST epochs and scan angles as a csv file.
GaiaDataDir = orvara/tests/data/gaia
# The path to all the Hipparcos (original reduction) intermediate data. Subfolders (as used on the original CD) are automatically handled.
Hip1DataDir = orvara/tests/data/hip1
# The path to all the Hipparcos (Floor van Leeuwen/second reduction) intermediate data. Subfolders

## Take note of the setting above in mcmc_settings:
    use_epoch_astrometry = True

This uses a precise treatment for the hipparcos and gaia absolute astrometry. In most cases, setting it to False is entirely fine and will result in nearly identical fits to the True case.

For this tutorial fit, everything will run fine. But for new fits, if you have issues with use_epoch_astrometry = True, simply set it to False and proceed with your fits. You can trouble shoot setting it to True it later.

If it is set to True, then the directory paths in the config file: GaiaDataDir, Hip1DataDir, and Hip2DataDir, have to exist. Orvara will not make them for you. Orvara will, however, download the appropriate Intermediate Data File for your source into those folders. If downloading, for whatever reason, does not work, then you can download the files manually and place them there (but it is a bit tricky -- see the documentation for HTOF).

Next, let's load the data sets needed for orbit fitting. There are three data types in general: RV, relative astrometry, or/and absolute astrometry. HD 159062 has all. Example data files are located in orvara/tests/data. Let's display them here:

In [8]:
ls orvara/tests/data

Gl758_RV.dat         HD159062_relAST.txt  gaia/
Gl758_relAST.txt     HD4747_RV.dat        hip1/
HD159062_RV.dat      HD4747_relAST.txt    hip2/


This is the RV data, it has four columns: Epoch[JD], RV[m/s], RV_err[m/s] and InstrumentID

In [14]:
print('This is what an orvara RV data input file looks like:')
display_file('orvara/tests/data/HD159062_RV.dat')

This is what an orvara RV data input file looks like:
# Instrument: HIRES on Keck
# Published in Hirsch, L. A., Ciardi, D. R., Howard, A. W., et al. 2019, ApJ, 878, 50
# File format:
# Epoch[JD]   RV[m/s] RV_err[m/s] InstrumentID
2452832.91668	89.22	1.62	0
2453074.09155	83.23	1.68	0
2453196.81631	76.88	1.58	0
2453430.15779	65.68	0.93	0
2453547.94513	60.98	0.84	0
2453604.84622	57.70	1.03	0
2453807.13748	50.30	1.00	0
2453926.45063	43.33	0.57	0
2453927.88530	44.96	0.83	0
2454248.03230	31.71	1.44	0
2454249.95458	32.00	1.49	0
2454252.04380	32.81	1.02	0
2454255.93510	32.28	1.01	0
2454277.86337	31.99	1.33	0
2454278.90748	32.22	1.22	0
2454279.94547	31.42	1.29	0
2454285.90800	30.58	1.34	0
2454294.90713	30.60	1.30	0
2454634.43562	19.74	1.03	0
2454635.94255	19.00	1.43	0
2454636.93316	17.04	1.53	0
2454638.33247	18.76	0.99	0
2454640.38633	14.57	1.06	0
2454641.94950	19.55	1.47	0
2454644.08325	15.11	1.52	0
2454688.88580	17.38	1.47	0
2454689.89920	9.37	1.58	0
2454957.13845	3.77	1.67	0
2455401.78282	-1

This is the relative astrometry file for HD 159062. It has seven columns: Date      , Sep (arcsec) , Err_sep  ,    PA (degrees),  Err_PA  Corr_Sep_PA, and  PlanetID. Planet ID depends on how many planets are in the system, 0 denotes the innermost planet, the second innermost is denoted 1 and so on.

In [15]:
print('This is what an orvara relative astrometry data input file looks like:')
display_file('orvara/tests/data/HD159062_relAST.txt')

This is what an orvara relative astrometry data input file looks like:
# Instrument: NIRC2 on Keck
# Published in Hirsch, L. A., Ciardi, D. R., Howard, A. W., et al. 2019, ApJ, 878, 50
##
# File format
#   Date       Sep  Err_sep      PA  Err_PA  Corr_Sep_PA  PlanetID  
 2012.48     2.594    0.014   298.6     0.1            0         0
 2014.78     2.637    0.014   299.5     0.1            0         0
 2018.32     2.673    0.003  301.27    0.04            0         0


For absolute astrometry from the HGCA, make sure that the HGCA file (DR2 or eDR3) is specified. We are using Gaia eDR3 version of the HGCA.

To use intermediate astrometry, make sure that the parameter use_epoch_astrometry in the config file is set to True.

We first recommend running fits with use_epoch_astrometry set to False, then turning it on later.

Make sure that the directories for the Gaia and Hipparcos intermediate data, set in the .ini config file, are valid directories. You will need one directory each for Hip1, Hip2, and Gaia.

orvara will download any file you need automatically (and save them to the directories you specified), but:

You may download the files by hand if you want, and simply put them in the data directory, with filenames containing the hip ID. The Gaia eDR3 observations are publicly accessible via the Gaia Observation Forecast Tool (https : //gaia.esac.esa.int/gost/index.jsp). Gaia files are .csv files queried from GOST. The Hip1 and Hip2 data are (if not, soon) avaliable publically from ESA.

Here are those files, with filenames containing HIP85653 are the ones we want for HD 159062. Remember, orvara should download these files automatically in normal use.

In [20]:
ls orvara/tests/data/gaia/

HIP003850.csv  HIP085653.csv  HIP095319.csv


In [21]:
ls orvara/tests/data/hip1/

003850.txt  085653.txt  95319.txt


In [22]:
ls orvara/tests/data/hip2/

HIP003850.d  HIP085653.d  HIP095319.d


### Now, let's run fit_orbit and plot_orbit commands in the root directory

We are going to run an example fit to HD 159062. Verifty that the RV data, relative astrometry data and absolute astrometry data are loaded in correctly to ensure a suceesful fit. Progress bar shows the progress of MCMC. 

In [24]:
!fit_orbit --output-dir orvara/tests/chains orvara/tests/config_HD159062.ini

Loading RV data from file orvara/tests/data/HD159062_RV.dat
Loaded data from 1 RV instruments.
Loading astrometric data for 1 planets
Loaded 3 relative astrometric data points from file orvara/tests/data/HD159062_relAST.txt
Loading absolute astrometry data for Hip 85653
Recognized a 5-parameter fit in Gaia for Hip 85653
Not using companion proper motion from Gaia.
/home/gmbrandt/envs/orvara_new/lib/python3.8/site-packages/htof/parse.py:371: UserWarning: For source 085653. The DVD IAD does not indicate which observation epochs were rejected for the final solution. htof will attempt to find which epochs to reject in order to reproduce the catalog parameters. However, if this source also has some corrupted residuals (see Brandt et al. 2021, Section 4), then this will fail. We recommend you switch to using the IAD from the Java tool, since that version of the IAD indicates rejected epochs with negative uncertainties.
  warnings.warn(f"For source {self.meta['star_id']}. The DVD IAD does not

Once you have the MCMC chain output by fit_orbit, feed it to McmcDataFile= in the config file. You can set a burn-in phase for the chain. After that, you are ready to go on to plot the orbits. plot_orbit will generate a suite of 7 plots relavant to RV and Astrometry, and a plot of the MCMC chain to breifly check convergence, and two files containting the beststep and posterior parameters. 

In [31]:
!plot_orbit --output-dir orvara/tests/plots/ orvara/tests/config_HD159062.ini

Generating plots for target HD159062B
Generating diagnostic plots to check convergence
Plotting Astrometry orbits, your plot is generated at orvara/tests/plots/
Plotting RV orbits, your plot is generated at orvara/tests/plots/
Plotting RV, your plot is generated at orvara/tests/plots/
Plotting Separation, your plot is generated at orvara/tests/plots/
Plotting Position Angle, your plot is generated at orvara/tests/plots/
Plotting Proper Motions, your plot is generated at orvara/tests/plots/
Plotting Corner plot, your plot is generated at orvara/tests/plots/
Saving beststep parameters to orvara/tests/plots/
Saving posterior parameters to orvara/tests/plots/


### Great! Your plots are generated in the default or specified output directory.

You can see the whole sets of plots in the output directory.

In [28]:
ls orvara/tests/plots/

astrometric_orbit_HD159062B.pdf       posterior_params_HD159062B.txt
astrometric_prediction_HD159062B.pdf  Proper_Motions_HD159062B.pdf
beststep_params_HD159062B.txt         relsep_OC_HD159062B.pdf
Corner_HD159062B.pdf                  RV_OC_HD159062B_InstAll.pdf
Diagnostic_HD159062B.png              RV_orbit_HD159062B.pdf
PA_OC_HD159062B.pdf


## For instance, here is the visual orbit.

In [32]:
from IPython.display import HTML
HTML('<iframe src=./orvara/tests/plots/astrometric_orbit_HD159062B.pdf width=700 height=350></iframe>')

/home/gmbrandt/envs/orvara_new/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## We can also inspect the posteriors and chisquareds from the fit:

In [29]:
print('Here are the chisquared values of the model with respect to the data (for all the input data types):')
display_file('orvara/tests/plots/beststep_params_HD159062B.txt')

Here are the chisquared values of the model with respect to the data (for all the input data types):
plx_ML  0.04618727907
pmra_ML  0.1695171527
pmdec_ML  0.08816530131
chisq_sep  0.7060916553
chisq_PA  2.141646391
chisq_H  4.077306854
chisq_HG  0.1247828772
chisq_G  0.2462001314
RV_ZP_0_ML  -1479.148349


In [30]:
print('Here are the posteriors with +- 1 sigma confidence intervals:')
display_file('orvara/tests/plots/posterior_params_HD159062B.txt')

Here are the posteriors with +- 1 sigma confidence intervals:
Jit (m/s)         ${1.26}_{-0.29}^{+0.33}$
Mpri (solar)         ${0.802}_{-0.050}^{+0.048}$
Msec (solar)         ${0.6057}_{-0.0069}^{+0.0077}$
a (AU)         ${56.8}_{-6.4}^{+4.4}$
sqesinw         ${-0.207}_{-0.087}^{+0.13}$
sqecosw         ${0.26}_{-0.19}^{+0.20}$
inclination (deg)         ${61.6}_{-2.9}^{+1.7}$
ascending node (deg)         ${134.5}_{-1.2}^{+2.2}$
mean longitude (deg)         ${147.3}_{-7.9}^{+5.7}$
parallax (mas)         ${46.1855}_{-0.0042}^{+0.0042}$
period (yrs)         ${361}_{-60}^{+44}$
argument of periastron (deg)         ${322}_{-47}^{+17}$
eccentricity         ${0.123}_{-0.077}^{+0.14}$
semimajor axis (mas)         ${2625}_{-296}^{+203}$
T0 (JD)         ${2515071}_{-9010}^{+13409}$
mass ratio         ${0.755}_{-0.044}^{+0.051}$
